In [1]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt

In [4]:
date = '20190715'
ww_fresh = xr.open_dataset('/ocean/cdonaldson/scrubbers/daily_obs/scrubber_obs_freshcalc_{}.nc'.format(date))
ww_fresh

<xarray.Dataset>
Dimensions:    (depth: 1, y: 898, x: 398)
Coordinates:
  * depth      (depth) float64 6.0
    time       datetime64[ns] ...
  * y          (y) int64 0 1 2 3 4 5 6 7 8 ... 890 891 892 893 894 895 896 897
  * x          (x) int64 0 1 2 3 4 5 6 7 8 ... 390 391 392 393 394 395 396 397
Data variables:
    washwater  (depth, y, x) float32 ...

## Goal
Transform ww_fresh into ww_est by adding salt and temperature, and replacing the stand-in depth with a static estimated depth for the model.

### Salt
Assume that the water at the outflow is the same salinity as the water at the intake, and that there is little delay in processing time within the scrubber system. Since we are doing daily runs, we can use the day-averaged salinity over the domain instead of hourly salinity. This means that these results will most likely represent open-loop scrubber systens.

### Temperature
A little bit trickier, since there isn't much literature about the temperature at the outflow and the guidelines (60C?) seem to be much more generous than any real values. Discussions of temperature tended to be buried within discussions of pH. There are two case studies summarized in the EPA report from 2011, "Exhaust Gas Scrubber Washwater Effluent" (inst. both references) and a modelling study from Ristea et al., "Computational Fluid Dynamics Simulation Approach for Scrubber Wash Water pH Modelling." Based on these, I am going to make the temperature at the outfall a function of the temperature of the intake water, with just an addition of X degrees.

Show work for number of degrees, but I'm thinking +10 or +7.5.

### Depth
This one is also tricky, since we already know that we will not be able to exactly represent the depth. Ships will sit higher or lower in the water based on their cargo load, and we don't have that information. One solution could be to pick one average depth, but I think it may be better to pick a range of depth to distribute the effluent outfall over since we have fine, 1m resolution in the model. Now, the question becomes one of "what range" and I hope would be a better representation for us to do time-averaged studies on. Does this then impact the assumption I am about to make where I use the surface salinity and temperature as my starting intake values?